In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import osmnx as ox
import pandas as pd
import os
from pyspark.sql.functions import *
from pyspark.sql.functions import col, regexp_replace, split

In [2]:
spark = SparkSession.Builder().getOrCreate()

In [3]:
usacc_df = spark.read.csv("./Data/US_Accidents/US_Accidents_Dec19.csv", header=True)
usacc_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- TMC: string (nullable = true)
 |-- Severity: string (nullable = true)
 |-- Start_Time: string (nullable = true)
 |-- End_Time: string (nullable = true)
 |-- Start_Lat: string (nullable = true)
 |-- Start_Lng: string (nullable = true)
 |-- End_Lat: string (nullable = true)
 |-- End_Lng: string (nullable = true)
 |-- Distance(mi): string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: string (nullable = true)
 |-- Temperature(F): string (nullable = true)
 |-- Wind_Chill(F): string (nullable = 

In [4]:
usacc_df.createOrReplaceTempView("usacc")

In [5]:
street_sqlDF = spark.sql("SELECT DISTINCT(CONCAT(City,', ',State,', ',Country,'A')) AS Address FROM usacc")
add_list = [row['Address'] for row in street_sqlDF.collect()]
add_list

['Lockbourne, OH, USA',
 'Greenwood, CA, USA',
 'Santa Monica, CA, USA',
 'Stanton, CA, USA',
 'Yucca Valley, CA, USA',
 'Ormond Beach, FL, USA',
 'Sidell, FL, USA',
 'Highland, IL, USA',
 'Riverview, MI, USA',
 'Luna Pier, MI, USA',
 'Fair Lawn, NJ, USA',
 'Irvington, NY, USA',
 'Hadley, MA, USA',
 'Springfield Gardens, NY, USA',
 'Pleasantville, NY, USA',
 'Berkeley Heights, NJ, USA',
 'Tenafly, NJ, USA',
 'Oakville, CT, USA',
 'Slingerlands, NY, USA',
 'Grand Rapids, OH, USA',
 'Mc Lean, VA, USA',
 'Breinigsville, PA, USA',
 'Sparrows Point, MD, USA',
 'Spring City, PA, USA',
 'Kittanning, PA, USA',
 'Snohomish, WA, USA',
 'Dupont, WA, USA',
 'Butler, GA, USA',
 'Wooster, OH, USA',
 'Albers, IL, USA',
 'Alger, MI, USA',
 'Ashley, OH, USA',
 'Fish Camp, CA, USA',
 'Harwich, MA, USA',
 'Wausau, WI, USA',
 'Union Pier, MI, USA',
 'Overton, NV, USA',
 'Kings Beach, CA, USA',
 'Hope Mills, NC, USA',
 'Big Bar, CA, USA',
 'Gardiner, OR, USA',
 'Golden Valley, AZ, USA',
 'Strawberry Plains

In [33]:
i = 1
for addr in add_list:
    try:
        graph = ox.graph_from_place(addr, network_type='drive')
        nodes, streets = ox.graph_to_gdfs(graph)
        streets = streets.drop('geometry', axis=1)
        streets.to_csv('./Data/Road Data/Road_Data_'+str(i)+'.csv')
        i+=1
    except Exception:
        continue

TopologyException: Input geom 0 is invalid: Self-intersection at or near point -91.669807967654165 45.30289788036567 at -91.669807967654165 45.30289788036567


In [4]:
road_folder = './Data/Road Data'
road_data = os.listdir(road_folder)
abspathev = []
for i in range(len(road_data)):
    abspathev.append(os.path.abspath(road_folder+'/'+road_data[i]))
road_df = spark.read.csv(abspathev, header=True)
road_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- u: string (nullable = true)
 |-- v: string (nullable = true)
 |-- key: string (nullable = true)
 |-- osmid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- highway: string (nullable = true)
 |-- oneway: string (nullable = true)
 |-- length: string (nullable = true)
 |-- lanes: string (nullable = true)
 |-- maxspeed: string (nullable = true)
 |-- junction: string (nullable = true)
 |-- access: string (nullable = true)
 |-- bridge: string (nullable = true)
 |-- ref: string (nullable = true)
 |-- tunnel: string (nullable = true)
 |-- service: string (nullable = true)
 |-- width: string (nullable = true)
 |-- area: string (nullable = true)



In [7]:
road_df.describe().toPandas()

,summary,_c0,u,v,key,osmid,name,highway,oneway,length,lanes,maxspeed,junction,access,bridge,ref,tunnel,service,width,area
0,count,29225788,29225788,29225788,29225788,29225788,25464791,27520279,27976066,27176153,9218668,5037419,1975417,630443,175824,54330,13075,2828,1971,66
1,mean,17123.689275307137,7.446330202602341E8,7.445663950593959E8,0.015069157416730731,1.1456927273000199E8,7.094438863897746,4.827061494239516,306.6842534620282,1909927.0138307575,141.64545687296672,131.80018555270564,117.55614258126252,67.06493806363144,9.885247753361757,9.576203966005671,1736.909263022508,9.449606299212597,11.364779874213836,None
2,stddev,34950.86378050069,1.5133765444075043E9,1.5131118432165954E9,0.12353329935197903,1.9755109496206072E8,313.3444506858292,119.91123096575451,556.5227123473394,6.186381000000144E9,919.3646031196254,517.0340661461936,191.1609827072154,330.7298299480643,21.119466972651377,4.427630890317207,9487.342160855851,5.537557538713542,4.9168547418551825,None
3,min,0,100000061,100000061,0,10000001,C Bar C Ranch Rd,"""""Alden's Oak""""]""","""""Amanda's Way""""]""","""""Benford's Road""""]""","""""St. Patrick's Place""""]""","""""Bearse's Way""""]""","""""Preacher's Brook Road""""]""","""""North O'Connor Road""""]""","""""28'""""]""",0,"""[""""18'""""","""""12'""""","""""20'""""","""""22'""""]"""
4,max,99999,99999909,99999909,9,"[9999976, 9990692]",Южная улица,❤️ Over Pocket,yt,yes,yes,yes,yes,yes,||no,yes|yes|yes|no|yes|yes,yes,yes,yes,residential


In [8]:
usacc_df.describe().toPandas()

,summary,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,count,2974335,2974335,2246264,2974335,2974335,2974335,2974335,2974335,728071,...,2974335,2974335,2974335,2974335,2974335,2974335,2974242,2974242,2974242,2974242
1,mean,None,None,207.83163198982845,2.360190092911525,None,None,36.493605002031636,-95.4262537385967,37.58087071324656,...,None,None,None,None,None,None,None,None,None,None
2,stddev,None,None,20.32958633116194,0.5414733263296426,None,None,4.918848998696759,17.218805933372185,5.00475695787924,...,None,None,None,None,None,None,None,None,None,None
3,min,A-1,Bing,200.0,1,2015-03-09 07:00:00,2016-02-08 06:37:08,24.555269,-100.000206,24.57011,...,False,False,False,False,False,False,Day,Day,Day,Day
4,max,A-999999,MapQuest-Bing,406.0,4,2020-01-01 00:50:55,2020-10-31 23:59:00,49.002201,-99.988158,49.075,...,True,True,True,True,True,False,Night,Night,Night,Night


In [9]:
road_df.createOrReplaceTempView("road")

In [10]:
road_sqlDF = spark.sql("SELECT name, highway, oneway, length, lanes, maxspeed FROM road")
road_sqlDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- highway: string (nullable = true)
 |-- oneway: string (nullable = true)
 |-- length: string (nullable = true)
 |-- lanes: string (nullable = true)
 |-- maxspeed: string (nullable = true)



In [11]:
road_sqlDF = road_sqlDF.withColumn("name",split(regexp_replace(col("name"), r"(^\[)|(\]$)|(')", ""), ","))
road_sqlDF = road_sqlDF.withColumn("highway",split(regexp_replace(col("highway"), r"(^\[)|(\]$)|(')", ""), ","))
road_sqlDF = road_sqlDF.withColumn("oneway",split(regexp_replace(col("oneway"), r"(^\[)|(\]$)|(')", ""), ","))
road_sqlDF = road_sqlDF.withColumn("length",split(regexp_replace(col("length"), r"(^\[)|(\]$)|(')", ""), ","))
road_sqlDF = road_sqlDF.withColumn("lanes",split(regexp_replace(col("lanes"), r"(^\[)|(\]$)|(')", ""), ","))
road_sqlDF = road_sqlDF.withColumn("maxspeed",split(regexp_replace(col("maxspeed"), r"(^\[)|(\]$)|(')", ""), ","))
road_sqlDF.show(truncate=False)

+---------------------------+--------------+-------+---------+-------+--------+
|name                       |highway       |oneway |length   |lanes  |maxspeed|
+---------------------------+--------------+-------+---------+-------+--------+
|[South Grenoble]           |[residential] |[False]|[94.741] |null   |null    |
|[East Rembrandt Avenue]    |[residential] |[False]|[190.412]|null   |null    |
|[South Grenoble]           |[residential] |[False]|[87.108] |null   |null    |
|[South Sunview]            |[residential] |[False]|[26.871] |null   |null    |
|[East Granite Avenue]      |[residential] |[False]|[35.399] |null   |null    |
|[South Sunview]            |[residential] |[False]|[26.871] |null   |null    |
|[East Granite Avenue]      |[residential] |[False]|[56.218] |null   |null    |
|[South Sunview]            |[residential] |[False]|[39.618] |null   |null    |
|[East La Salle Street]     |[residential] |[False]|[27.46]  |[2]    |[25 mph]|
|[East La Salle Street]     |[residentia

In [12]:
road_sqlDF = road_sqlDF.select(explode(col("name")).alias("name"),col("highway"),col("oneway"),col("length"),col("lanes"),col("maxspeed"))
road_sqlDF = road_sqlDF.select(col("name"),explode(col("highway")).alias("highway"),col("oneway"),col("length"),col("lanes"),col("maxspeed"))
road_sqlDF = road_sqlDF.select(col("name"),col("highway"),explode(col("oneway")).alias("oneway"),col("length"),col("lanes"),col("maxspeed"))
road_sqlDF = road_sqlDF.select(col("name"),col("highway"),col("oneway"),explode(col("length")).alias("length"),col("lanes"),col("maxspeed"))
road_sqlDF = road_sqlDF.select(col("name"),col("highway"),col("oneway"),col("length"),explode(col("lanes")).alias("lanes"),col("maxspeed"))
road_sqlDF = road_sqlDF.select(col("name"),col("highway"),col("oneway"),col("length"),col("lanes"),explode(col("maxspeed")).alias("maxspeed"))
road_sqlDF = road_sqlDF.na.drop()
road_sqlDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- highway: string (nullable = true)
 |-- oneway: string (nullable = true)
 |-- length: string (nullable = true)
 |-- lanes: string (nullable = true)
 |-- maxspeed: string (nullable = true)



In [13]:
road_sqlDF = road_sqlDF.withColumn('name', trim(road_sqlDF.name))
road_sqlDF = road_sqlDF.withColumn('highway', trim(road_sqlDF.highway))
road_sqlDF = road_sqlDF.withColumn('oneway', trim(road_sqlDF.oneway))
road_sqlDF = road_sqlDF.withColumn('length', trim(road_sqlDF.length))
road_sqlDF = road_sqlDF.withColumn('lanes', trim(road_sqlDF.lanes))
road_sqlDF = road_sqlDF.withColumn('maxspeed', trim(road_sqlDF.maxspeed))

In [14]:
road_sqlDF.createOrReplaceTempView("road_df")
road_sqlDF = spark.sql("SELECT * FROM road_df WHERE highway LIKE 'motorway' OR highway LIKE 'trunk' OR highway LIKE 'primary' OR highway LIKE 'secondary' OR highway LIKE 'tertiary' OR highway LIKE 'unclassified' OR highway LIKE 'residential'")
road_sqlDF = spark.sql("SELECT * FROM road_df WHERE oneway LIKE 'True' OR oneway LIKE 'False'")
road_sqlDF = spark.sql("SELECT * FROM road_df WHERE length LIKE '1%' OR length LIKE '2%' OR length LIKE '3%' OR length LIKE '4%' OR length LIKE '5%' OR length LIKE '6%' OR length LIKE '7%' OR length LIKE '8%' OR length LIKE '9%' OR length LIKE '0%'")
road_sqlDF = spark.sql("SELECT * FROM road_df WHERE lanes == 1 OR lanes == 2 OR lanes == 3 OR lanes == 4 OR lanes == 5 OR lanes == 6 OR lanes == 7 OR lanes == 8 OR lanes == 9 OR lanes == 10 OR lanes == 11 OR lanes == 12 OR lanes == 13 OR lanes == 14 OR lanes == 15 OR lanes == 16")
road_sqlDF = spark.sql("SELECT * FROM road_df WHERE maxspeed LIKE '%mph'")
road_sqlDF = road_sqlDF.dropDuplicates(['name'])

In [15]:
road_sqlDF = road_sqlDF.withColumn('name', regexp_replace('name', 'Road', ''))
road_sqlDF = road_sqlDF.withColumn('name', regexp_replace('name', 'Avenue', ''))
road_sqlDF = road_sqlDF.withColumn('name', regexp_replace('name', 'Street', ''))
road_sqlDF.show(truncate=False)

+--------------------+------------+------+------------------+------+--------+
|name                |highway     |oneway|length            |lanes |maxspeed|
+--------------------+------------+------+------------------+------+--------+
|13th  Southeast     |residential |False |361.875           |2     |25 mph  |
|Bannock             |residential |False |185.14            |1     |25 mph  |
|Barron Way          |unclassified|False |341.033           |2     |35 mph  |
|Beacon              |residential |False |267.16499999999996|2     |25 mph  |
|Brescher Drive      |residential |False |296.289           |2     |25 mph  |
|Broadway            |tertiary    |True  |408.55899999999997|GA 124|45 mph  |
|Cadillac Place      |unclassified|False |231.79100000000003|2     |25 mph  |
|Camino Tassajara    |secondary   |True  |77.726            |2     |45 mph  |
|Canyon              |residential |False |303.291           |1     |25 mph  |
|Constitution Highway|primary     |False |91.911            |VA 

In [16]:
usacc_sqlDF = spark.sql("SELECT Severity, `Distance(mi)` AS Distance, Street, Side, `Temperature(F)` AS Temperature, `Wind_Chill(F)` AS Wind_Chill, `Humidity(%)` AS Humidity, `Pressure(in)` AS Pressure, `Visibility(mi)` AS Visibility, Wind_Direction, `Wind_Speed(mph)` AS Wind_Speed, `Precipitation(in)` AS Precipitation, Weather_Condition, Amenity, Bump, Crossing, Give_Way, Junction, No_Exit, Railway, Roundabout, Station, Stop, Traffic_Calming, Traffic_Signal, Turning_Loop, Sunrise_Sunset, Civil_Twilight, Nautical_Twilight, Astronomical_Twilight FROM usacc")
usacc_sqlDF.show()

+--------+--------+--------------------+----+-----------+----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|Severity|Distance|              Street|Side|Temperature|Wind_Chill|Humidity|Pressure|Visibility|Wind_Direction|Wind_Speed|Precipitation|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Civil_Twilight|Nautical_Twilight|Astronomical_Twilight|
+--------+--------+--------------------+----+-----------+----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+

In [17]:
usacc_sqlDF = usacc_sqlDF.withColumn('Street', regexp_replace('Street', 'Rd', ''))
usacc_sqlDF = usacc_sqlDF.withColumn('Street', regexp_replace('Street', 'Ave', ''))
usacc_sqlDF = usacc_sqlDF.withColumn('Street', regexp_replace('Street', 'St', ''))
usacc_sqlDF.show()

+--------+--------+--------------------+----+-----------+----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|Severity|Distance|              Street|Side|Temperature|Wind_Chill|Humidity|Pressure|Visibility|Wind_Direction|Wind_Speed|Precipitation|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Civil_Twilight|Nautical_Twilight|Astronomical_Twilight|
+--------+--------+--------------------+----+-----------+----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+

In [18]:
road_sqlDF.createOrReplaceTempView("road_df")
usacc_sqlDF.createOrReplaceTempView("usacc_df")
merged_sqlDF = spark.sql("SELECT * FROM usacc_df ua, road_df r WHERE ua.Street = r.name")

In [19]:
#print(merged_sqlDF.count())
merged_sqlDF = merged_sqlDF.drop('Street')
merged_sqlDF = merged_sqlDF.drop('name')
merged_sqlDF = merged_sqlDF.na.drop()
print(merged_sqlDF.count())

68983


In [20]:
merged_sqlDF.describe().toPandas()

,summary,Severity,Distance,Side,Temperature,Wind_Chill,Humidity,Pressure,Visibility,Wind_Direction,...,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,highway,oneway,length,lanes,maxspeed
0,count,68983,68983,68983,68983,68983,68983,68983,68983,68983,...,68983,68983,68983,68983,68983,68983,68983,68983,68983,68983
1,mean,2.153371120420973,0.12293930384905613,None,61.106320397779186,59.38936549584682,68.65288549352739,29.483081628807106,8.730162358842028,None,...,None,None,None,None,None,None,None,281.50233408231026,6.77955876984058,None
2,stddev,0.38019703972630237,0.9725041299737098,None,19.660199427775833,22.349363753097588,20.968244919212086,0.8711249345752929,3.1463374767002175,None,...,None,None,None,None,None,None,None,710.6026699159021,89.24707472962673,None
3,min,1,0.0,L,-0.0,-0.1,10.0,20.36,0.0,CALM,...,False,Day,Day,Day,Day,living_street,False,10.142,(616),10 mph
4,max,4,9.975,R,99.0,99.0,99.0,31.08,9.0,West,...,False,Night,Night,Night,Night,unclassified,True,99.98400000000001,yes,75 mph


In [71]:
rdac_df = merged_sqlDF.toPandas()
rdac_df['maxspeed'] = rdac_df['maxspeed'].str.replace("mph", "")
rdac_df['maxspeed'] = rdac_df['maxspeed'].str.replace(";", "")
rdac_df['maxspeed'] = rdac_df['maxspeed'].str.replace(" ", "")
rdac_df['maxspeed'] = rdac_df['maxspeed'].apply(pd.to_numeric)
rdac_df = rdac_df[rdac_df.maxspeed != 4035]
rdac_df = rdac_df.reset_index(drop=True)

In [72]:
rdac_df = rdac_df[rdac_df['lanes'].isin(['1','2','3','4','5','6','7','8','9','10'])]
rdac_df = rdac_df.reset_index(drop=True)
rdac_df

,Severity,Distance,Side,Temperature,Wind_Chill,Humidity,Pressure,Visibility,Wind_Direction,Wind_Speed,...,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,highway,oneway,length,lanes,maxspeed
0,2,0.0,R,51.0,51.0,78.0,29.76,10.0,W,8.0,...,False,Night,Night,Day,Day,residential,False,455.119,2,25
1,3,1.15999996662,R,79.0,79.0,58.0,29.84,10.0,SW,9.0,...,False,Night,Night,Night,Night,residential,False,455.119,2,25
2,2,0.0,L,48.0,43.0,83.0,29.86,8.0,E,12.0,...,False,Day,Day,Day,Day,residential,False,455.119,2,25
3,2,0.0,L,67.0,67.0,40.0,29.64,10.0,VAR,3.0,...,False,Day,Day,Day,Day,residential,False,455.119,2,25
4,2,0.0,R,56.0,56.0,85.0,30.13,10.0,CALM,0.0,...,False,Day,Day,Day,Day,residential,False,455.119,2,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57312,2,0.0,R,78.0,78.0,76.0,29.82,10.0,E,5.0,...,False,Day,Day,Day,Day,tertiary,False,1560.524,2,45
57313,2,0.0,L,73.0,73.0,96.0,29.52,7.0,ESE,5.0,...,False,Day,Day,Day,Day,tertiary,False,1560.524,2,45
57314,2,0.0,R,66.0,66.0,86.0,29.55,10.0,ESE,5.0,...,False,Day,Day,Day,Day,tertiary,False,1560.524,2,45
57315,2,0.0,R,50.0,50.0,35.0,30.09,10.0,W,3.0,...,False,Day,Day,Day,Day,tertiary,False,1560.524,2,45


In [75]:
pd.set_option('display.precision',12)
rdac_df['Distance'] = rdac_df['Distance'].apply(float)
rdac_df.dtypes

Severity                  object
Distance                 float64
Side                      object
Temperature               object
Wind_Chill                object
Humidity                  object
Pressure                  object
Visibility                object
Wind_Direction            object
Wind_Speed                object
Precipitation             object
Weather_Condition         object
Amenity                   object
Bump                      object
Crossing                  object
Give_Way                  object
Junction                  object
No_Exit                   object
Railway                   object
Roundabout                object
Station                   object
Stop                      object
Traffic_Calming           object
Traffic_Signal            object
Turning_Loop              object
Sunrise_Sunset            object
Civil_Twilight            object
Nautical_Twilight         object
Astronomical_Twilight     object
highway                   object
oneway    

In [77]:
rdac_df.to_csv('./Data/Merged Data/US_Accidents_Road.csv', index = False, float_format='%.12f')